In this tutorial, we cover the `pyphs.Evaluation` object for the numerical evaluation of `pyphs.Core` objects. The core object from the previous tutorial on [`pyphs.Core`](https://afalaize.github.io/pyphs/posts/tutos/pyphs-core/) is used as an example.

<!-- TEASER_END -->

The corresponding Python script `pyphs-evaluation.py` can be found in the tutorials at `\pyphs\tutorials\`

In [1]:
# Support for Python 2.x and Python 3.x
from __future__ import division, print_function, absolute_import

# retrieve the pyphs.Core of a nonlinear RLC from the previous tutorial
from pyphs.tutorials.core import core

# load the pyphs.Evaluation class in the current namespace
from pyphs import Evaluation

# instantiate a pyphs.Evaluation object associated with a pyphs.PHSCore
evals = Evaluation(core)

Remove Inverse of Parameters...
Build numerical evaluations...


The object `evals` contains all numerical versions for all the function from the original `core`. The arguments are $\mathbf x, \, \delta \mathbf x, \, \mathbf w, \, \mathbf u, \, \mathbf p$, as returned by the function `core.args`:

In [2]:
evals.args()

[xL, xK, xM, dxL, dxK, dxM, v1]

In [3]:
# values for arguments
x = [5e-4, 2e-2]
dx = [5e-3, 2e-1]
w = [5e-3, ]
u = [1.5, ]

# collect all arguments
args = x + dx + w + u

# retrieve values for arguments of the Hamiltonian function
args_H = [args[i] for i in evals.H_inds]

# numerical evaluations of the Hamiltonian function
H = evals.H(*args_H)
# print "H(xL, xC) = 3947.84176069"
print('{}{} = {}'.format('H', evals.H_args, H))

H(xL, xK, xM) = 8000.000669258373


In [4]:
# print "H(xL, xC) = 3947.84176069"
print('{}{} = {}'.format('H', evals.H_args, H))

# Same as above for a list of PHScore expressions and matrices
for name in ('dxH', 'z', 'Jxx', 'Rxx'):

    # retrieve the function to evaluate
    func = getattr(evals, name)

    # retrieve the ordered list of symbols for arguments
    func_args = getattr(evals, name + '_args')

    # retrieve the indices of symbols for arguments in the global args vector
    func_inds = getattr(evals, name + '_inds')

    # retrieve the values for arguments
    func_args_vals = [args[i] for i in func_inds]

    # evaluate the function with arguments values
    func_eval = func(*func_args_vals)

    # print e.g. "z() = "
    print('\n{}{} = \n'.format(name, func_args), func_eval)


# Notice function are vectorized:
x1, x2, x3 = range(0, 3), range(4, 7), range(10, 13) 
evals.H(x1, x2, x3)

H(xL, xK, xM) = 8000.000669258373

dxH(xL, xK, xM) = 
 [  4.54545455e-02   8.00000000e+05   2.63157895e-01]

z() = 
 []

Jxx(xK,) = 
 [[ 0.          0.          2.98880424]
 [ 0.          0.          1.        ]
 [-2.98880424 -1.          0.        ]]

Rxx() = 
 [[ 5.7    0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.406]]


array([  3.20002632e+08,   5.00003230e+08,   7.20003971e+08])

In [5]:
print("\nVectorized (not parallel) evaluation:\nx1={}, x2={}, x3={}".format(x1, x2, x3))
print("H(x1, x2, x3)={}".format(evals.H(x1, x2, x3)))


Vectorized (not parallel) evaluation:
x1=range(0, 3), x2=range(4, 7), x3=range(10, 13)
H(x1, x2, x3)=[  3.20002632e+08   5.00003230e+08   7.20003971e+08]
